In [4]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

from google.colab import files
files.upload()
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d iarunava/cell-images-for-detecting-malaria
!ls

Saving kaggle.json to kaggle (2).json
cell-images-for-detecting-malaria.zip: Skipping, found more recently modified local copy (use --force to force download)
 cell-images-for-detecting-malaria.zip	'kaggle (2).json'   Malaria
'kaggle (1).json'			 kaggle.json	    sample_data


In [5]:
from zipfile import ZipFile
zip_ref = ZipFile('/content/cell-images-for-detecting-malaria.zip', 'r')
zip_ref.extractall('/content/Malaria')
zip_ref.close()
import os
from PIL import Image
import numpy as np

img_rows=44
img_cols=44

path1='/content/Malaria/cell_images/Parasitized'
path2='/content/Malaria/cell_images/Uninfected'

listing = os.listdir(path1)
num_samples = len(listing)
print(num_samples)
images=[]
for file in listing:
  
    try:
      im=Image.open(path1 + '/' + file)
      images.append(np.asarray(im.resize((img_rows, img_cols))))
    except:
      continue
images1 = np.array(images)

listing = os.listdir(path2)
num_samples = len(listing)-1
print(num_samples)
images=[]
for file in listing:
    try:
      im=Image.open(path2 + '/' + file)
      images.append(np.asarray(im.resize((img_rows, img_cols))))
    except:
      continue
images2 = np.array(images)

13780
13779


In [6]:
import numpy as np
from PIL import Image
import random
import os




# KERAS
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D as Conv2D
from keras.layers.convolutional import MaxPooling2D, AveragePooling2D
from keras.optimizers import SGD, RMSprop, Adam
from keras.losses import categorical_crossentropy
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras.utils import np_utils
from keras import regularizers


!pip install keract
from keract import get_activations

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os
# SKLEARN
from sklearn.utils import shuffle

Using TensorFlow backend.


In [7]:
label=np.zeros((num_samples*2,2))
label[:num_samples,0]=1
label[num_samples:,1]=1

data=np.zeros((num_samples*2,44,44,3))
data[:num_samples,:,:,:]=images1
data[num_samples:num_samples*2,:,:,:]=images2

data=data/255.0

quee=np.random.permutation(num_samples*2)
label=label[quee,:]
data=data[quee,:,:,:]

print(np.shape(label),np.shape(data))


length=len(label)
train_count=int(length*0.8)
valid_count=int(length*0.1)
test_count=length-train_count-valid_count
train_data=data[:train_count,:,:,:]
train_label=label[:train_count,:]
valid_data=data[train_count:train_count+valid_count,:,:,:]
valid_label=label[train_count:train_count+valid_count,:]
test_data=data[train_count+valid_count:,:,:,:]
test_label=label[train_count+valid_count:,:]





num_classes=2
batch_size=64
epochs=100
dropout=0.2

model = Sequential()
model.add(Conv2D(128, kernel_size=(3, 3), strides=(1, 1),
                 activation='relu',
                 input_shape=(44,44,3)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(dropout))

model.add(Conv2D(256, kernel_size=(3, 3), strides=(1, 1),
                 activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(dropout))

model.add(Conv2D(512, kernel_size=(3, 3), strides=(1, 1),
                 activation='relu'))
model.add(Conv2D(512, kernel_size=(3, 3), strides=(1, 1),
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(dropout))


model.add(Conv2D(512, kernel_size=(2, 2), strides=(1, 1),
                 activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(256, activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(128, activation='relu'))
model.add(Dropout(dropout))
#model.add(Dense(64, activation='sigmoid'))
#model.add(Dropout(dropout))
#model.add(Dense(10, activation='sigmoid'))
#model.add(Dense(2, activation='sigmoid'))
model.add(Dense(num_classes, activation=tf.nn.softmax,activity_regularizer=regularizers.l2(0.0001)))
model.summary()

adam=optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001, amsgrad=False)
model.compile(loss=categorical_crossentropy,
              optimizer=adam,
              metrics=['accuracy'])


model.fit(train_data, train_label,
          batch_size=64,
          epochs=epochs,
          verbose=1,
          validation_data=(valid_data, valid_label))


score = model.evaluate(test_data, test_label, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

(27558, 2) (27558, 44, 44, 3)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 42, 42, 128)       3584      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 40, 40, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 20, 128)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 20, 20, 128)       512       
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 20, 128)       0         
_________________________________________________________

In [8]:
released=get_activations(model, train_data[1:3,:])
for release in released:
    print(release)

conv2d_1/Relu:0
conv2d_2/Relu:0
max_pooling2d_1/MaxPool:0
batch_normalization_1/cond/Merge:0
dropout_1/cond/Merge:0
conv2d_3/Relu:0
conv2d_4/Relu:0
max_pooling2d_2/MaxPool:0
batch_normalization_2/cond/Merge:0
dropout_2/cond/Merge:0
conv2d_5/Relu:0
conv2d_6/Relu:0
max_pooling2d_3/MaxPool:0
batch_normalization_3/cond/Merge:0
dropout_3/cond/Merge:0
conv2d_7/Relu:0
batch_normalization_4/cond/Merge:0
dropout_4/cond/Merge:0
flatten_1/Reshape:0
dense_1/Relu:0
dropout_5/cond/Merge:0
dense_2/Relu:0
dropout_6/cond/Merge:0
dense_3/Relu:0
dropout_7/cond/Merge:0
dense_4/Softmax:0


In [12]:
length=len(test_label)
step=int(length/10)
test_outdata=np.zeros((length,2))
for i in range(10):
  try:
    test_outdata[i*step:(i+1)*step,:]=get_activations(model, test_data[i*step:(i+1)*step,:]).get("dense_4/Softmax:0","none")
  except:
    print("hello")
    test_outdata[i*step:,:]=get_activations(model, test_data[i*step:,:]).get("dense_4/Softmax:0","none")
                                         
print(np.shape(test_outdata))
y_test=test_label[:,0]
y_pred_test=np.zeros((length,1))
for i in range (length):
  if test_outdata[i,0]<test_outdata[i,1]:
    y_pred_test[i,0]=0
  else:
    y_pred_test[i,0]=1 
from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test,y_pred_test))  
print(classification_report(y_test,y_pred_test))  

(2757, 2)
[[1364   27]
 [  86 1280]]
              precision    recall  f1-score   support

         0.0       0.94      0.98      0.96      1391
         1.0       0.98      0.94      0.96      1366

   micro avg       0.96      0.96      0.96      2757
   macro avg       0.96      0.96      0.96      2757
weighted avg       0.96      0.96      0.96      2757

